# Reading and Loading data

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install datasets transformers seqeval conllu wandb

In [3]:
file = open("/content/gdrive/MyDrive/hashtagseg/conllpp_train.txt","r+") 

In [4]:
!wc -l /content/gdrive/MyDrive/hashtagseg/conllpp_train.txt

219553 /content/gdrive/MyDrive/hashtagseg/conllpp_train.txt


In [5]:
ner = []
separated = []
lines = file.readlines()

In [6]:
len(lines)

219553

In [7]:
sentence = ''
labels = ''
sentence_sep = ''
count = 0
x_fin = 219553
x = 0
for i,line in enumerate(lines):

    if i< 10:
      print(line.split())
      print(line.lower().split())

    x += 1
    if x == x_fin:
        break
    if line == '-DOCSTART- -X- -X- O\n':
        continue
    if line != '\n':
        word = line.split()[0]
        sentence += word
        sentence_sep += '_' + word
        label = line.split()[-1].strip('\n').split('-')[0]
        if label == 'B':
            labels += 'B'
            for i in range(len(word)-1):
                labels += 'B'
        elif label == 'I':
            for i in range(len(word)):
                labels += 'I'
        elif label == 'O':
            for i in range(len(word)):
                labels += 'O'
        count += 1
    else:
        if count != 0:
            ner.append((sentence, labels, count))
            # separated.append((sentence_sep[1:], labels))
            sentence = ''
            labels = ''
            count = 0

['-DOCSTART-', '-X-', '-X-', 'O']
['-docstart-', '-x-', '-x-', 'o']
[]
[]
['EU', 'NNP', 'B-NP', 'B-ORG']
['eu', 'nnp', 'b-np', 'b-org']
['rejects', 'VBZ', 'B-VP', 'O']
['rejects', 'vbz', 'b-vp', 'o']
['German', 'JJ', 'B-NP', 'B-MISC']
['german', 'jj', 'b-np', 'b-misc']
['call', 'NN', 'I-NP', 'O']
['call', 'nn', 'i-np', 'o']
['to', 'TO', 'B-VP', 'O']
['to', 'to', 'b-vp', 'o']
['boycott', 'VB', 'I-VP', 'O']
['boycott', 'vb', 'i-vp', 'o']
['British', 'JJ', 'B-NP', 'B-MISC']
['british', 'jj', 'b-np', 'b-misc']
['lamb', 'NN', 'I-NP', 'O']
['lamb', 'nn', 'i-np', 'o']


In [8]:
len(ner)

14040

In [9]:
len(separated)

0

In [10]:
ner[0]

('EUrejectsGermancalltoboycottBritishlamb.',
 'BBOOOOOOOBBBBBBOOOOOOOOOOOOOBBBBBBBOOOOO',
 9)

In [13]:
ner[0]

('EUrejectsGermancalltoboycottBritishlamb.',
 'BBOOOOOOOBBBBBBOOOOOOOOOOOOOBBBBBBBOOOOO',
 9)

In [14]:
from tqdm import tqdm
from conllu.models import TokenList, Token



In [15]:
allsamples = []

for sample in tqdm(ner):

  assert len(list(sample[0])) == len(list(sample[1]))

  tl = TokenList()

  for ind,(c,a) in enumerate(zip (list(sample[0].lower()),list(sample[1]))):
    # print(ind,c,a)
    tl.append(Token(id =ind,form=c,tag= a))

  # print(tl)

  allsamples.append(tl)

#     sentence2 = TokenList([
# ...    Token(id=(1, "-", 2), form="It's"),
# ...    Token(id=1, form="It"),
# ...    Token(id=2, form="is"),
# ... ])
    # print(c,'\t',a)
  # break

100%|██████████| 14040/14040 [00:01<00:00, 7246.46it/s]


In [16]:
with open('test.conll', 'w') as f:
...     f.writelines([sentence.serialize() + "\n" for sentence in allsamples])

# do not execute - proto

In [ ]:
# def preproc(hashtag,annotation):

#   tokenized_tokens = tokenizer.convert_ids_to_tokens(tokenizer(hashtag)['input_ids'])

#   print(tokenized_tokens[1:-1])

#   splits = [len(token[2:]) if token[:2]=='##' else len(token) for token in tokenized_tokens[1:-1]]
  
#   print(splits)

#   splits = np.cumsum(splits).tolist()

#   print(splits)

#   splits = [0] + splits[:-1]

#   print(splits)

#   for i,j in zip(splits,splits[1:]+[None]):
#     print(i,j)
#   parts = [annotation[i:j] for i,j in zip(splits, splits[1:]+[None])]


#   print(parts)

#   # [annotation for ind,val in zip(splits,splits[1:])]


#   # [if token[0] == '#' for token in tokenized_tokens]


In [ ]:
# def preproc2(hashtag,annotations):
#   tokenized_input = tokenizer(list(hashtag), is_split_into_words=True)
#   print(len(tokenized_input['input_ids']),len(annotations))


In [ ]:
# hashtag = "EUrejectsGermancalltoboycottBritishlamb."
# annotations = "BBOOOOOOOBBBBBBOOOOOOOOOOOOOBBBBBBBOOOOO"

# preproc2(hashtag,annotations)

# # tokenized_input = tokenizer(hashtag, is_split_into_words=False)
# # tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
# # print(tokens)

In [ ]:
# annotations[37:None]
# hashtag[37:None]

# Preprocessing, tokenisation

In [17]:
model_checkpoint = "bert-base-multilingual-uncased"
batch_size = 8

from datasets import load_dataset, load_metric
import numpy as np

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
from datasets import load_dataset, load_metric

In [19]:
datasets_ner = load_dataset('loadDataset.py',)


Reusing dataset universal_dependencies (/root/.cache/huggingface/datasets/universal_dependencies/default/2.7.0/24afa0285828f614697f46e61ee766e87abd555a5313531b89893e1f84108b1e)


In [20]:
import datasets
datasets_ner = datasets.Dataset.train_test_split(datasets_ner['train'],test_size=0.20,shuffle=True,seed=1234)


Loading cached split indices for dataset at /root/.cache/huggingface/datasets/universal_dependencies/default/2.7.0/24afa0285828f614697f46e61ee766e87abd555a5313531b89893e1f84108b1e/cache-0730f9a49173340c.arrow and /root/.cache/huggingface/datasets/universal_dependencies/default/2.7.0/24afa0285828f614697f46e61ee766e87abd555a5313531b89893e1f84108b1e/cache-2fe57948ea6595c9.arrow


In [21]:
datasets_ner

DatasetDict({
    train: Dataset({
        features: ['idx', 'tags', 'tokens'],
        num_rows: 11232
    })
    test: Dataset({
        features: ['idx', 'tags', 'tokens'],
        num_rows: 2808
    })
})

In [22]:
datasets_ner['train'][0]

{'idx': '3822',
 'tags': [2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 'tokens': "B A S E B A L L - M A J O R L E A G U E S T A N D I N G S A F T E R F R I D A Y ' S G A M E S ."}

In [23]:
assert len(datasets_ner['train'][0]['tags']) == len(datasets_ner['train'][0]['tokens'].split())
# len(datasets_ner['train'][0]['tags']), len(datasets_ner['train'][0]['text'].split())


In [24]:
datasets_ner["train"].features[f"tags"]

Sequence(feature=ClassLabel(num_classes=3, names=['B', 'I', 'O'], names_file=None, id=None), length=-1, id=None)

In [25]:
label_list = datasets_ner["train"].features[f"tags"].feature.names
label_list

['B', 'I', 'O']

In [26]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [27]:
label_all_tokens = True

In [28]:
def tokenize_and_align_labels(examples):

    # print(type(examples), examples['tokens'])

    for ind,tl in enumerate(examples['tokens']):
      examples["tokens"][ind] = tl.split()      

    # print(examples['tokens'])

    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [29]:
tokenize_and_align_labels(datasets_ner['test'][:5])

{'input_ids': [[101, 165, 147, 160, 146, 147, 160, 144, 160, 147, 155, 147, 156, 125, 122, 122, 123, 126, 127, 125, 102], [101, 123, 119, 153, 150, 143, 154, 151, 146, 144, 157, 163, 154, 143, 155, 151, 113, 155, 157, 160, 157, 145, 145, 157, 114, 128, 131, 124, 122, 119, 127, 126, 102], [101, 129, 119, 167, 157, 161, 150, 151, 153, 143, 165, 143, 124, 129, 131, 124, 126, 119, 123, 130, 128, 102], [101, 143, 154, 144, 143, 156, 151, 143, 143, 161, 153, 147, 146, 149, 160, 147, 147, 145, 147, 157, 156, 165, 147, 146, 156, 147, 161, 146, 143, 167, 162, 157, 147, 166, 158, 154, 143, 151, 156, 165, 150, 167, 151, 162, 165, 143, 161, 146, 147, 158, 157, 160, 162, 151, 156, 149, 155, 157, 160, 147, 143, 154, 144, 143, 156, 151, 143, 156, 151, 155, 155, 151, 149, 160, 143, 156, 162, 161, 117, 148, 157, 160, 147, 151, 149, 156, 155, 151, 156, 151, 161, 162, 147, 160, 162, 160, 151, 162, 143, 156, 161, 150, 147, 150, 163, 161, 143, 151, 146, 119, 102], [101, 154, 157, 156, 146, 157, 156, 122, 1

In [30]:
tokenized_datasets = datasets_ner.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/universal_dependencies/default/2.7.0/24afa0285828f614697f46e61ee766e87abd555a5313531b89893e1f84108b1e/cache-85728f5e5c825241.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/universal_dependencies/default/2.7.0/24afa0285828f614697f46e61ee766e87abd555a5313531b89893e1f84108b1e/cache-17745fb84fef5a96.arrow


# Training

In [31]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint a

In [32]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [33]:
args = TrainingArguments(
    f"test-ner-hashtag",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    save_total_limit = 1,
    save_steps = 500,
    eval_steps = 500,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [34]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [35]:
metric = load_metric("seqeval")


In [36]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [37]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [38]:
import wandb

!wandb login

#319ea6f7049796b3b25f922660a9f9595b7fc9c2


wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [39]:
wandb.init(project="hashtag-seg-ner",name="run3-uncased-lower", config={
    "learning_rate": 2e-5,
    "architecture": "mbert",
    "dataset": "random",
})
config = wandb.config

wandb: Currently logged in as: prakod (use `wandb login --relogin` to force relogin)


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.497500,0.399723,0.704248,0.534176,0.607534,0.861159
1000,0.401500,0.353221,0.729633,0.606604,0.662455,0.875046


In [ ]:
trainer.save_model('/content/gdrive/MyDrive/hashtagseg/rum3-uncased-lower')

# Prediction

In [19]:
from transformers import AutoModel,AutoConfig,AutoModelForTokenClassification
config = AutoConfig.from_pretrained('/content/gdrive/MyDrive/hashtagseg/config.json')



model = AutoModelForTokenClassification.from_pretrained('/content/gdrive/MyDrive/hashtagseg',config=config)

In [23]:
model.to('cuda')

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [24]:
def predict_nertags(sentence):

  # print(sentence.split())

  # print(tokenizer.tokenize(list(sentence), is_split_into_words = True))

  # print(tokenizer(list(sentence), is_split_into_words=True, return_tensors='pt'))

  tokenized_sentence = tokenizer(list(sentence), is_split_into_words=True, return_tensors='pt')

  # print(tokenized_sentence)

  outputs = model(**tokenized_sentence.to('cuda'))

  # print(outputs.keys())

  preds = np.argmax(outputs['logits'].cpu().detach().numpy(), axis=2).squeeze()

  mask = []
  prev_id = None
  for ind,id in enumerate(tokenized_sentence.word_ids()):
    
    if id is None:
      mask.append(-100)
    elif id == prev_id:
      mask.append(-100)
    elif id != prev_id:
      mask.append(id)
    prev_id = id


  true_preds = [
      label_list[p] for (p, l) in zip(preds, mask) if l != -100
  ]

  # print(true_preds)

  print(sentence)
  print('*'*70)
  for word,pred in zip(list(sentence),true_preds,):
    print(word,'\t\t-',pred)

In [25]:
hashtag = 'RaniLakshmiBai'

predict_nertags(hashtag)

RaniLakshmiBai
**********************************************************************
R 		- B
a 		- B
n 		- B
i 		- B
L 		- I
a 		- I
k 		- I
s 		- I
h 		- I
m 		- I
i 		- I
B 		- I
a 		- I
i 		- I


In [35]:
hashtag = '#Gazaunderattack'

predict_nertags(hashtag)

#Gazaunderattack
**********************************************************************
# 		- O
G 		- B
a 		- B
z 		- B
a 		- B
u 		- O
n 		- O
d 		- O
e 		- O
r 		- O
a 		- O
t 		- O
t 		- O
a 		- O
c 		- O
k 		- O


# rest of code


In [ ]:
separated[0]

('EU_rejects_German_call_to_boycott_British_lamb_.',
 'BIOOOOOOOBIIIIIOOOOOOOOOOOOOBIIIIIIOOOOO')

In [ ]:
ner[0][0]

'EUrejectsGermancalltoboycottBritishlamb.'

In [ ]:
ner[0][1]

'BIOOOOOOOBIIIIIOOOOOOOOOOOOOBIIIIIIOOOOO'

In [ ]:
len(ner[0][0])

40

In [ ]:
len(ner[0][1])

40

In [ ]:
file = open("BTC.conll","r+") 

In [ ]:
ner = []
separated = []
lines = file.readlines()

In [ ]:
len(lines)

17612

In [ ]:
sentence = ''
labels = ''
sentence_sep = ''
count = 0
for line in lines:
    if line == '-DOCSTART- -X- -X- O\n':
        continue
    if line != '\n':
        word = line.split()[0]
        sentence += word
        sentence_sep += '_' + word
        label = line.split()[-1].strip('\n').split('-')[0]
        if label == 'B':
            labels += 'B'
            for i in range(len(word)-1):
                labels += 'I'
        elif label == 'I':
            for i in range(len(word)):
                labels += 'I'
        elif label == 'O':
            for i in range(len(word)):
                labels += 'O'
        count += 1
    else:
        if count != 0:
            ner.append((sentence, labels, count))
            separated.append((sentence_sep[1:], labels))
            sentence = ''
            sentence_sep = ''
            labels = ''
            count = 0

In [ ]:
len(ner)

1000

In [ ]:
ner[90]

('listeningto\'"\'TimeisnotEnough-BRENDALEE\'"\'♫http://t.co/nQYdQoO8',
 'OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIIIIIOOOOOOOOOOOOOOOOOOOOOOOO',
 13)

In [ ]:
separated[90]

('listening_to_\'"\'_Time_is_not_Enough_-_BRENDA_LEE_\'"\'_♫_http://t.co/nQYdQoO8',
 'OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIIIIIOOOOOOOOOOOOOOOOOOOOOOOO')